In [0]:
#!/usr/bin/env python3
from math import inf as infinity
from random import choice
import platform
import time
from os import system

HUMAN = -1
AI = +1
board = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]


In [0]:
# Function to heuristic evaluation of state.
def calc_score(state):
    if check_for_winner(state, AI):
        score = +1
    elif check_for_winner(state, HUMAN):
        score = -1
    else:
        score = 0

    return score

In [0]:
def check_for_winner(state, player):
    """
    This function tests if a specific player wins. Possibilities:
    * Three rows    [X X X] or [O O O]
    * Three cols    [X X X] or [O O O]
    * Two diagonals [X X X] or [O O O]
    """
    win_state = [
        [state[0][0], state[0][1], state[0][2]],
        [state[1][0], state[1][1], state[1][2]],
        [state[2][0], state[2][1], state[2][2]],
        [state[0][0], state[1][0], state[2][0]],
        [state[0][1], state[1][1], state[2][1]],
        [state[0][2], state[1][2], state[2][2]],
        [state[0][0], state[1][1], state[2][2]],
        [state[2][0], state[1][1], state[0][2]],
    ]
    if [player, player, player] in win_state: # checks if  [x x x] or [o o o] is in any row or colum or diagonal
        return True
    else:
        return False

In [0]:
# check who wins 
def check_if_game_over(state):
    return check_for_winner(state, HUMAN) or check_for_winner(state, AI)


In [0]:
def empty_cells(state):
    """
    Each empty cell will be added into cells' list
    """
    cells = []

    for x, row in enumerate(state):
        #print('x: ',x)
        #print('row: ',row)
        for y, cell in enumerate(row):
            #print('y: ',y)
            #print('cell: ',cell)
            if cell == 0: 
              cells.append([x, y])
              #print(cells)
    return cells

In [0]:
#A move is valid if the chosen cell is empty

def check_if_valid_move(x, y):
    if [x, y] in empty_cells(board):
        return True
    else:
        return False

In [0]:
# Set the move on board, if the coordinates are valid

def set_move(x, y, player):

    if check_if_valid_move(x, y):
        board[x][y] = player
        return True
    else:
        return False

In [0]:
# recursively call minimax, to evaluate all possible moves, and make the best move for winning position

def minimax(state, depth, player):
    if player == AI:
      # initialize coordinates of best move, and score of best move
        best = [-1, -1, -infinity] # -infiniy because, we need AI to choose max value
    else:
      # initialize coordinates of best move, and score of best move
        best = [-1, -1, +infinity] # +infiniy because, human will choose min value for AI

    if depth == 0 or check_if_game_over(state): # are there no empty cells remaining? or is there already a winner?
        score = calc_score(state)
        return [-1, -1, score]
 
    for cell in empty_cells(state): # for the remaining empty cells, check for best possible move
        x, y = cell[0], cell[1] # take first available empty cell
        state[x][y] = player   # mark as +1 if AI's turn, -1 if human's turn
        score = minimax(state, depth - 1, -player) #-player, because players turns alternate. This is where/how we expand the tree( recursively calling minimax)
        state[x][y] = 0 # wipe out changes, make cell empty again
        score[0], score[1] = x, y # get calculated score
 
        if player == AI:
            if score[2] > best[2]:
                best = score  # max value
        else:
            if score[2] < best[2]:
                best = score  # min value for human player

    return best

In [0]:
def clear_console():
    """
    Clears the console
    """
    os_name = platform.system().lower() #this is from : import platform. gives name of os
    #print(os_name)
    if 'windows' in os_name:
        system('cls')
    else:
        system('clear')

In [0]:
import os
import platform
from os import system
clear_console()

In [0]:
def display_board(state, ai_choice, human_choice):
    """
    Print the board on console
    :param state: current state of the board
    """
    print('----------------')
    for row in state:
        print('\n----------------')
        for cell in row:
            if cell == +1:
                print('|', ai_choice, '|', end='')
            elif cell == -1:
                print('|', human_choice, '|', end='')
            else:
                print('|', ' ', '|', end='')
    print('\n----------------')

In [0]:
# For the first move, choose random coordinate. Else call minimax function

def ai_turn(ai_choice, human_choice):

    depth = len(empty_cells(board))
    if depth == 0 or check_if_game_over(board): # if no more moves left or any player won, return to main
        return

    clear_console()
    print('Computer turn [{}]'.format(ai_choice))
    display_board(board, ai_choice, human_choice)

    if depth == 9:  #for ai's first move, choose any position randomly
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
    else:
        move = minimax(board, depth, AI)
        x, y = move[0], move[1]

    set_move(x, y, AI)
    time.sleep(1)


In [0]:
# up to human, to make a valid move

def human_turn(ai_choice, human_choice):

    depth = len(empty_cells(board))
    #print(depth)
    if depth == 0 or check_if_game_over(board):
        return

    # Dictionary of valid moves
    move = -1
    moves = {
        1: [0, 0], 2: [0, 1], 3: [0, 2],
        4: [1, 0], 5: [1, 1], 6: [1, 2],
        7: [2, 0], 8: [2, 1], 9: [2, 2],
    }

    clear_console()
    print('Human turn [{}]'.format(human_choice))
    display_board(board, ai_choice, human_choice)

    while (move < 1 or move > 9):
        try:
            move = int(input('Enter position (1..9): '))
            board_position = moves[move]
            try_move = set_move(board_position[0], board_position[1], HUMAN)

            if try_move == False:
                print('Invalid move')
                move = -1
        except KeyboardInterrupt:
            print('Bye')
            exit()
        except:
            print('Invalid choice')


In [0]:
# Main function that calls all functions
def main():

    clear_console() # function written to clear console
    human_choice = '' # X or O
    ai_choice = '' # X or O
    first = ''  # if human is the first

    # Human chooses X or O to play
    while human_choice != 'O' and human_choice != 'X':
        try:
            print('')
            human_choice = input('Choose X or O\nChosen: ').upper()
        except KeyboardInterrupt:
            print('Bye')
            exit()
        except:
            print('Invalid choice')

    # Setting computer's choice
    if human_choice == 'X':
        ai_choice = 'O'
    else:
        ai_choice = 'X'

    # Human may start first
    clear_console()
    while first != 'Y' and first != 'N':
        try:
            first = input('First to start?[y/n]: ').upper()
        except KeyboardInterrupt:
            print('Bye')
            exit()
        except:
            print('Invalid choice')

    # Main loop of this game
    while len(empty_cells(board)) > 0 and not check_if_game_over(board): # while moves remain and no one has won
        if first == 'N':
            ai_turn(ai_choice, human_choice) # computer makes move if human chose to not play first
            first = ''

        human_turn(ai_choice, human_choice)
        ai_turn(ai_choice, human_choice)

    # Game over message
    if check_for_winner(board, HUMAN):
        clear_console()
        print('Human turn [{}]'.format(human_choice))
        display_board(board, ai_choice, human_choice)
        print('YOU WIN!')
    elif check_for_winner(board, AI):
        clear_console()
        print('Computer turn [{}]'.format(ai_choice))
        display_board(board, ai_choice, human_choice)
        print('YOU LOSE!')
    else:
        clear_console()
        display_board(board, ai_choice, human_choice)
        print('DRAW!')

    exit()



In [18]:

if __name__ == '__main__':
    main()


Choose X or O
Chosen: x
First to start?[y/n]: n
Computer turn [O]
----------------

----------------
|   ||   ||   |
----------------
|   ||   ||   |
----------------
|   ||   ||   |
----------------
Human turn [X]
----------------

----------------
|   ||   ||   |
----------------
|   ||   ||   |
----------------
| O ||   ||   |
----------------
Enter position (1..9): 9
Computer turn [O]
----------------

----------------
|   ||   ||   |
----------------
|   ||   ||   |
----------------
| O ||   || X |
----------------
Human turn [X]
----------------

----------------
| O ||   ||   |
----------------
|   ||   ||   |
----------------
| O ||   || X |
----------------
Enter position (1..9): 4
Computer turn [O]
----------------

----------------
| O ||   ||   |
----------------
| X ||   ||   |
----------------
| O ||   || X |
----------------
Human turn [X]
----------------

----------------
| O ||   || O |
----------------
| X ||   ||   |
----------------
| O ||   || X |
---------------